## Scraping Definitions

In [ ]:
# 有道词典 definitions 
defs = driver.find_element_by_xpath('//*[@id="phrsListTab"]/div[2]/ul/p').text
# LINE Dict. definitions
defs = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[3]/ol').text

# I added this comment to test pushing back to Github
# And this one:
# Did it work?

## The Importer

In [22]:
# Import necessary modules
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re
from io import StringIO
import json
import urllib.request
import pandas as pd
import numpy as np
import playsound
from numpy import random
from datetime import datetime
from langdetect import detect
import winsound
from playsound import playsound
from gtts import gTTS
import os
# Define the current date and time
now = datetime.now().strftime("%Y-%m-%d_%H.%M.%S")
startTime1 = time.time()

# DEFINE FUNCTIONS

def convert(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%dh %02dm %02ds" % (hour, minutes, seconds)

def convertTTS(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%d hours %02d minutes %02d seconds" % (hour, minutes, seconds)

# Define the Sentence Scraping function
def scrapeSentences(language):
    words = vocab['Foreign']
    n = 0
    o = words.shape[0]
    # Create master DataFrame
    w = {}
    concatList = []
    # Run the scraper based on the language
    if language == '日本語':
        # Not done
        # See CSV_Gen [ALL].ipynb
        pass
    elif language == '中文':
        # Define URL for Chinese words
        url_zh = 'https://dict.naver.com/linedict/zhendict/dict.html#/cnen/home'
        # Define the driver for Selenium to use
        driver = webdriver.Chrome('chromedriver.exe')
        # Open the specfied URL
        driver.get(url_zh)
        # Find the radio group and select "Examples"
        examples_radio = driver.find_element_by_xpath('//*[@id="ac_form"]/fieldset/ul/li[2]/span').click()
        # Find the Search bar
        search = driver.find_element_by_xpath('//*[@id="ac_input"]')
        # Scrape sentences for each word present in the vocabulary list
        for word in words:
            # Type the word into the Search Box
            search.send_keys(word)
            search.send_keys(Keys.RETURN)
            # Sleep to avoid errors
            time.sleep(1.5)
            # Find the sentences examples
            examples = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]')
            # Define an object with all the example sentences
            unfiltered_sentences = examples.text
            # Clean the string of text that was just scraped
            pass1 = unfiltered_sentences.replace("more", "")
            # Finalize the DataFrame
            myList = pass1.split("\n")
            myList = myList[::2]
            # Drop the 拼音 be creating a dictionary
            myDict = dict(zip(myList[::2], myList[1::2]))
            foreign = myDict.keys()
            english = myDict.values()
            df = pd.DataFrame(columns = ['English', 'Foreign', 'Word', 'Hint', 'Cloze'])
            df['English'] = english
            df['Foreign'] = foreign
            df['Word'] = word
            wordsClean = words.tolist()
            wordsClean = str(wordsClean).strip("[]")
            wordsClean = wordsClean.replace("'", "")
            df['Hint'] = wordsClean
            df['Hint'] = df['Hint'].str.strip('[]')
            df['Hint'] = df['Hint'].str.replace("'", "")
            # Create the Cloze sentences
            df['Cloze'] = df['Foreign'].str.replace(word, '{{c1::' + word + '}}')
            # Remove examples that don't contain a period
            df['English'] = df['English'].map(str)
            df1 = df[df['English'].str.contains('\.', na = False)]
            # Reduce the DataFrame to only 3 sentences
            df2 = df1.iloc[:3]
            # Create the final DataFrame for the word
            w[word] = df2
            # Add the DataFrame to the list of all word DataFrames
            concatList.append(w[word])
            # Sleep to avoid errors
            time.sleep(0.5)
            n += 1
            o -= 1
            print("Elapsed time: {} for {} words ({} remaining)".format(convert(time.time() - startTime1), n, o))
            # Exit the FOR loop
        # Quit the browser
        driver.quit()
        # Concatenate each word's DataFrame to the final one with all the sentences
        dfm = pd.concat(concatList, ignore_index = True)
        # Fix formating issues
        dfm['Cloze'] = dfm['Cloze'].str.replace('Name:.*', '')
        dfm['Cloze'] = dfm['Cloze'].str.replace('\n', ',')
        dfm['Cloze'] = dfm['Cloze'].str.replace('\d', '')
        # return the created DataFrame
        return dfm
    elif language == '한국어':
        # Define URL for Korean words
        url_ko = 'https://en.dict.naver.com/#/search?range=example&shouldSearchVlive=false&query=%ED%9B%84%ED%9A%8C&exampleLevel=exist:1&haveTrans=exist:1'
        # Define the driver for Selenium to use
        driver = webdriver.Chrome('chromedriver.exe')
        # Open the specfied URL
        driver.get(url_ko)
        # Scrape sentences for each word present in the vocabulary list
        for word in words:
            # Find the Search bar
            search = driver.find_element_by_xpath('//*[@id="ac_input"]')
            # Sleep to avoid errors
            time.sleep(1)
            # Type the word into the Search Box
            search.clear()
            # Sleep to avoid errors
            time.sleep(1)
            search.send_keys(word)
            search.send_keys(Keys.RETURN)
            # Sleep to avoid errors
            time.sleep(2)
            # Find the button titled "Examples" and click it
            example_button = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[1]/div/div/a[4]')
            example_button.send_keys(Keys.RETURN)
            # Sleep to avoid errors
            time.sleep(1)
            # Find the "Level 1" radio button and select it
            level_1 = driver.find_element_by_xpath('//*[@id="searchPage_example"]/div[1]/div/span[2]').click()
            # Sleep to avoid errors
            time.sleep(1)
            # Find the "Have Translations" check box and check it
            have_translations = driver.find_element_by_xpath('//*[@id="searchPage_example"]/div[1]/div/span[5]/label').click()
            # Sleep to avoid errors
            time.sleep(2.5)
            # Find the "View more examples" link and click it if it is displayed
            if driver.find_element_by_xpath('//*[@id="searchPage_example_more"]').is_displayed():
                driver.find_element_by_xpath('//*[@id="searchPage_example_more"]').click()
            else:
                pass
            # Sleep to avoid errors
            time.sleep(2)
            # Find the sentence examples
            examples = driver.find_element_by_xpath('//*[@id="searchPage_example"]/div[2]')
            # Define an object with all the example sentences
            unfiltered_sentences = examples.text
            pass1 = unfiltered_sentences.replace("발음듣기", "")
            pass2 = pass1.replace("Oxford Advanced Learner's English-Korean Dictionary", "")
            pass3 = pass2.replace("절대어휘 5100", "")
            pass4 = pass3.replace("국제영어대학원대학교 신어사전", "")
            pass5 = pass4.replace("DARAKWON", "")
            pass6 = pass5.replace("Friendict Level English-Korean Dictionary", "")
            pass7 = pass6.replace("Web Collection", "")
            pass8 = pass7.replace("YBM All in All English-Korean Dictionary", "")
            pass9 = pass8.replace("Neungyule", "")
            pass10 = pass9.replace("User translation", "")
            pass11 = pass10.replace("Hansard", "")
            pass12 = pass11.replace("영어영작문대사전", "")
            pass13 = pass12.replace("English Hidden Card", "")
            pass14 = pass13.replace("TIMES CORE", "")
            # Create a Dictionary with which to create the final DataFrame
            myList = pass14.split("\n\n")
            # Remove the parentheitcal words from the list
            myList2 = []
            char_list = ['(', ')']
            for ele in myList:
                for idx in char_list:
                    if idx not in ele:
                        flag = 1
                    else:
                        flag = 0
                        break
                if(flag ==1):
                    myList2.append(ele)
            # Make dictionary from the new list
            myDict = dict(zip(myList2[::2], myList2[1::2]))
            english = myDict.keys()
            foreign = myDict.values()
            # Create a DataFrame from the scraped string
            df = pd.DataFrame(columns = ['English', 'Foreign', 'Word', 'Hint', 'Cloze'])
            df['English'] = english
            df['Foreign'] = foreign
            df['Word'] = word
            wordsClean = words.tolist()
            wordsClean = str(wordsClean).strip("[]")
            wordsClean = wordsClean.replace("'", "")
            df['Hint'] = wordsClean
            df['Hint'] = df['Hint'].str.replace("'", "")
            # Handle conjugations
            if "하다" in word:
                wordConj = word[:-2]
            elif "다" in word:
                wordConj = word[:-1]
            else:
                wordConj = word
            # Create the Cloze sentences    
            df['Cloze'] = df['Foreign'].str.replace(wordConj, '{{c1::' + wordConj + '}}')
            # Remove examples that don't contain a period
            df['English'] = df['English'].map(str)
            # Reduce the DataFrame to only 3 sentences
            df2 = df.iloc[:3]
            # Create the final DataFrame for the word
            w[word] = df2
            # Add the DataFrame to the list of all word DataFrames
            concatList.append(w[word])
            n += 1
            o -= 1
            print("Elapsed time: {} for {} words ({} remaining)".format(convert(time.time() - startTime1), n, o))
            # Exit the FOR loop
        # Quit the browser
        driver.quit()
        # Concatenate each word's DataFrame to the final one with all the sentences
        dfm = pd.concat(concatList, ignore_index = True)
        # Remove the pesky fuckers that got through the cleaning passes filters
        dfm['Foreign'] = dfm['Foreign'].str.replace('\n.*', '')
        dfm['Cloze'] = dfm['Cloze'].str.replace('\n.*', '')
        # return the created DataFrame
        return dfm
        # Exit the IF-ELIF statement
# Define the Anki-Connect functions
def request(action, **params):
    return {'action': action, 'params': params, 'version': 6}
def invoke(action, **params):
    requestJson = json.dumps(request(action, **params)).encode('utf-8')
    response = json.load(urllib.request.urlopen(urllib.request.Request('http://localhost:8765', requestJson)))
    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')
    if 'error' not in response:
        raise Exception('response is missing required error field')
    if 'result' not in response:
        raise Exception('response is missing required result field')
    if response['error'] is not None:
        raise Exception(response['error'])
    return response['result']
# Define the three functions we'll be using
# Define Vocab Notes function
def addVocab(language):
    n = 0
    # Handle the unique 日本語
    if language == '日本語':
    # Loop through all the rows and create a note for each one
        for index, vocab in enumerate(vocabForeign):
            invoke('addNote', note = {
                        "deckName": '{}'.format(language),
                        "modelName": '(NEW) {}'.format(language),
                        "fields": {
                            "(Word) Foreign": vocab, 
                            "(Word) English": vocabEnglish[index],
                            "(Word) Foreign [漢字]": kanji
                            }  
                })
            n += 1
            print("{} has elapsed for {} Vocab words of {} ({} remaining).".format(convert(time.time() - startTime1), n, words.shape[0], words.shape[0] - n))
    # For 한국어 and 中文
    else:
        # Loop through all the rows and create a note for each one
        for index, vocab in enumerate(vocabForeign):
            invoke('addNote', note = {
                        "deckName": '{}'.format(language),
                        "modelName": '(NEW) {}'.format(language),
                        "fields": {
                            "(Word) Foreign": vocab, 
                            "(Word) English": vocabEnglish[index]
                            }  
                })
            n += 1
            print("{} has elapsed for {} Vocab words of {} ({} remaining).".format(convert(time.time() - startTime1), n, words.shape[0], words.shape[0] - n))
# Define Cloze Notes function
def addCloze(language):
    n = 0
    # Loop through all the rows and create a note for each one
    for index, sentence in enumerate(cloze):
        invoke('addNote', note = {
                    "deckName": '{}'.format(language),
                    "modelName": '(NEW) {} Cloze'.format(language),
                    "fields": {
                        "Text": sentence, 
                        "Tip": tip[index],
                        "English": english[index],
                        "full setence": fullSentence[index]
                        }  
            })
        n += 1
        print("{} has elapsed for {} Cloze notes of {} ({} remaining).".format(convert(time.time() - startTime1), n, dfm.shape[0], dfm.shape[0] - n))

# THE REAL WORK BEGINS

# Read CSVs into DataFrames
vocab = pd.read_excel(r'Flashcards.xlsx', sheet_name = '(Test) Chinese {Large}', header = 0)
vocab.columns = ['Foreign', 'English', 'Kanji', 'Language']
vocab['English'] = vocab['English'].replace('s/\([^)]*\)//', '')
# Retrieve the language from the vocab DataFrame then drop its column
language = vocab['Language'].loc[0:0].to_string(index = False, header = False)
vocab.drop(['Language'], inplace = True, axis = 1)
# Strip the leading space from the lang variable (Thank you shit Excel dropdown menu! /s)
language = language.lstrip()
# Format English Definitions 
if language == '中文':
    # CHINESE DEFINIIIONS FORMATTING
    # Add spaces after Numbers
    vocab['English'] = vocab['English'].replace("([1-9])", "\\1.) ", regex = True)
    # Remove empty paragraphs
    vocab['English'] = vocab['English'].replace('\n\n', '; ', regex = True)
    # Remove spaces before semicolons
    vocab['English'] = vocab['English'].replace(' ;', ',', regex = True)
    # Add a new line after semicolons
    vocab['English'] = vocab['English'].replace('(;)', '\\1\n', regex = True)
elif language == '한국어':
    # KOREAN DEFINITIONS FORMATTING
    # Remove empty paragraphs
    vocab['English'] = vocab['English'].replace('\n', ' ', regex = True)
    # Add a new line before a number
    vocab['English'] = vocab['English'].replace("([2-9])", "\n \\1", regex = True)
elif language == '日本語':
    pass
# Define the returned DataFrame from scrapeSentences()
words = vocab['Foreign']
print("Processing {} {} words for scraping...".format(words.shape[0], language))
dfm = scrapeSentences(language)
# Define the necessary variables
vocabForeign = vocab['Foreign'].to_list()
vocabEnglish = vocab['English'].to_list()
kanji = vocab['Kanji'].to_list()
# Define the necessary variables
cloze = dfm['Cloze'].to_list()
tip = dfm['Hint'].to_list()
english = dfm['English'].to_list()
fullSentence = dfm['Foreign'].to_list()
# Add the Notes
print("Processing {} {} Vocabulary...".format(words.shape[0], language))
addVocab(language)
print("Processing {} {} Clozes...".format(dfm.shape[0], language))
addCloze(language)
print("ALL PROCESSES COMPLETED! ELAPSED TIME: {}".format(convert(time.time() - startTime1)))
tts = gTTS(text = "All processes have completed in {}.".format(convertTTS(time.time() - startTime1)), lang = "en")
tts.save("tts.mp3")
playsound("tts.mp3")
os.remove("tts.mp3")

Processing 26 中文 words for scraping...
Elapsed time: 0h 00m 07s for 1 words (25 remaining)
Elapsed time: 0h 00m 10s for 2 words (24 remaining)
Elapsed time: 0h 00m 12s for 3 words (23 remaining)
Elapsed time: 0h 00m 15s for 4 words (22 remaining)
Elapsed time: 0h 00m 17s for 5 words (21 remaining)
Elapsed time: 0h 00m 20s for 6 words (20 remaining)
Elapsed time: 0h 00m 22s for 7 words (19 remaining)
Elapsed time: 0h 00m 25s for 8 words (18 remaining)
Elapsed time: 0h 00m 27s for 9 words (17 remaining)
Elapsed time: 0h 00m 30s for 10 words (16 remaining)
Elapsed time: 0h 00m 32s for 11 words (15 remaining)
Elapsed time: 0h 00m 35s for 12 words (14 remaining)
Elapsed time: 0h 00m 38s for 13 words (13 remaining)
Elapsed time: 0h 00m 40s for 14 words (12 remaining)
Elapsed time: 0h 00m 43s for 15 words (11 remaining)
Elapsed time: 0h 00m 45s for 16 words (10 remaining)
Elapsed time: 0h 00m 48s for 17 words (9 remaining)
Elapsed time: 0h 00m 50s for 18 words (8 remaining)
Elapsed time: 0h 0